Downloading the file to data/ directory and verifying the file

In [ ]:
import gdown

file_id = "1IyDLXcIGnrLhsvJ-CCRoOGNxQKCn7f7x"
url = f"https://drive.google.com/uc?id={file_id}"

output_path = "data/Restaurants_Train.xml"
gdown.download(url, output_path, quiet=False)

print("File downloaded. Checking contents:")
with open(output_path, 'r', encoding='utf-8') as f:
    print(f.read()[:500]) 

Downloading...
From (original): https://drive.google.com/uc?id=1IyDLXcIGnrLhsvJ-CCRoOGNxQKCn7f7x
From (redirected): https://drive.google.com/uc?id=1IyDLXcIGnrLhsvJ-CCRoOGNxQKCn7f7x&confirm=t&uuid=c6a9d8fc-cc1b-4a60-8788-6f0f7b5f37ad
To: /teamspace/studios/this_studio/data/Restaurants_Train.xml
100%|██████████| 1.24M/1.24M [00:00<00:00, 72.2MB/s]

File downloaded. Checking contents:
<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<sentences>
    <sentence id="3121">
        <text>But the staff was so horrible to us.</text>
        <aspectTerms>
            <aspectTerm term="staff" polarity="negative" from="8" to="13"/>
        </aspectTerms>
        <aspectCategories>
            <aspectCategory category="service" polarity="negative"/>
        </aspectCategories>
    </sentence>
    <sentence id="2777">
        <text>To be completely fair, the only redeeming factor 


Verifying the file and Loading and splitting into train/validation

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET

def parse_semeval(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    data = []
    for sentence in root.findall('.//sentence'):
        text = sentence.find('text').text
        for aspect_term in sentence.findall('.//aspectTerm'):
            aspect = aspect_term.get('term')
            sentiment = aspect_term.get('polarity')
            data.append({
                'sentence': text,
                'aspect': aspect,
                'sentiment': sentiment
            })
    return pd.DataFrame(data)
semeval_data = parse_semeval('data/Restaurants_Train.xml')
semeval_data = semeval_data[:200]
train_data = semeval_data[:160]
val_data = semeval_data[:40]
train_data.to_csv('data/semeval_train.csv', index=False)
val_data.to_csv('data/semeval_val.csv', index=False)
print(f"Loaded {len(semeval_data)} SemEval sentences (Train: {len(train_data)}, Val: {len(val_data)})")
print("First 5 rows of train data:")
print(train_data.head())
print("Sentiment distribution (train):")
print(train_data['sentiment'].value_counts())
print("Sentiment distribution (val):")
print(val_data['sentiment'].value_counts())

Loaded 200 SemEval sentences (Train: 160, Val: 40)
First 5 rows of train data:
                                            sentence   aspect sentiment
0               But the staff was so horrible to us.    staff  negative
1  To be completely fair, the only redeeming fact...     food  positive
2  The food is uniformly exceptional, with a very...     food  positive
3  The food is uniformly exceptional, with a very...  kitchen  positive
4  The food is uniformly exceptional, with a very...     menu   neutral
Sentiment distribution (train):
positive    99
neutral     30
negative    27
conflict     4
Name: sentiment, dtype: int64
Sentiment distribution (val):
positive    20
neutral     12
negative     8
Name: sentiment, dtype: int64
